In [1]:
!pip install -U transformers datasets evaluate optuna seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
###############################################################################
# Indic‑NER fine‑tuning on Naamapadam (+ optional custom JSON)
###############################################################################
import argparse, random, json, os, itertools, logging, collections
import numpy as np, torch
from datasets import load_dataset, concatenate_datasets, DatasetDict, Sequence, Value
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)

# ──────────────────────────────────────────────────────────────
# 0.  LOGGING
# ──────────────────────────────────────────────────────────────
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%H:%M:%S",
)
log = logging.getLogger(__name__)

# ──────────────────────────────────────────────────────────────
# 1.  ARGUMENTS & SEEDING
# ──────────────────────────────────────────────────────────────
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, default="ai4bharat/indic-bert")
parser.add_argument("--languages", nargs="+", default=["as"])
parser.add_argument("--custom_data_path", type=str, default="/content/drive/MyDrive/Yash_final_btp/naamapadam_proj/0.6/0.4/naamapadam_assamese.json",
                    help="Path to your extra JSON file (optional)")
parser.add_argument("--output_dir", type=str,
                    default="./indicner-finetuned-naamapadam")
parser.add_argument("--num_train_epochs", type=int, default=20)
parser.add_argument("--per_device_train_batch_size", type=int, default=32)
parser.add_argument("--per_device_eval_batch_size", type=int, default=32)
parser.add_argument("--learning_rate", type=float, default=1e-5)
parser.add_argument("--weight_decay",  type=float, default=0.01)
parser.add_argument("--warmup_steps",  type=int, default=500)
parser.add_argument("--seed", type=int, default=42)
args, _ = parser.parse_known_args()

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
    log.info("CUDA device: %s", torch.cuda.get_device_name(0))
else:
    log.info("CUDA not available – falling back to CPU")

lang = args.languages[0]          # single‑language run

# ──────────────────────────────────────────────────────────────
# 2.  CANONICAL LABEL SET  (Naamapadam has exactly 7)
# ──────────────────────────────────────────────────────────────
label_list = ["B-LOC","B-ORG","B-PER","I-LOC","I-ORG","I-PER","O"]
id2label   = {i:l for i,l in enumerate(label_list)}  # type: ignore
label2id   = {l:i for i,l in id2label.items()}
num_labels = len(label_list)
log.info("Canonical labels: %s", id2label)

# Helper – map *any* tag outside this list to “O”
def normalise_tag(tag: str) -> str:
    tag = str(tag)
    if tag in ("B-PERSON", "I-PERSON"):         # your PERSON alias
        tag = tag.replace("PERSON", "PER")
    return tag if tag in label2id else "O"

# ──────────────────────────────────────────────────────────────
# 3.  LOAD NAAMAPADAM (train / test)
# ──────────────────────────────────────────────────────────────
train_ref = load_dataset("ai4bharat/naamapadam", lang, split="train")
test_ref  = load_dataset("ai4bharat/naamapadam", lang, split="test")

# NEW  ➜  make their `ner_tags` column a plain int sequence
int_seq = Sequence(Value("int64"))
train_ref = train_ref.cast_column("ner_tags", int_seq)
test_ref  = test_ref.cast_column("ner_tags", int_seq)

# ──────────────────────────────────────────────────────────────
# 4.  OPTIONAL: LOAD + CLEAN YOUR CUSTOM JSON
# ──────────────────────────────────────────────────────────────
if args.custom_data_path:
    log.info("Loading custom JSON: %s", args.custom_data_path)
    custom = load_dataset("json", data_files=args.custom_data_path,
                          split="train")

    # (a) ensure tokens / ner_tags length match
    broken = [i for i, ex in enumerate(custom)
              if len(ex["tokens"]) != len(ex["ner_tags"])]
    if broken:
        log.warning("⚠️  %d rows have mismatching lengths – they’ll be dropped",
                    len(broken))
        custom = custom.select([i for i in range(len(custom)) if i not in broken])

    # (b) normalise & map to ints
    def _clean(batch):
        out = []
        for tags in batch["ner_tags"]:
            out.append([label2id[normalise_tag(t)] for t in tags])
        batch["ner_tags"] = out
        return batch

    custom = custom.map(_clean, batched=True)
    log.info("Custom set after cleaning: %d sentences", len(custom))
    train_all = concatenate_datasets([train_ref, custom])
else:
    train_all = train_ref

# quick label‑distribution print‑out
def label_hist(ds, name):
    flat = list(itertools.chain.from_iterable(ds["ner_tags"]))
    c = collections.Counter(flat)
    log.info("%s label distribution: %s",
             name, {id2label[k]: v for k,v in c.items()})
label_hist(train_all, "TRAIN")
label_hist(test_ref,  "TEST ")

# 5. TRAIN / DEV SPLIT ---------------------------------------------------------
split = train_all.train_test_split(train_size=0.75, seed=args.seed)
train_ds, dev_ds = split["train"], split["test"]
log.info("Train %d  |  Dev %d  |  Test %d", len(train_ds), len(dev_ds), len(test_ref))

# 6. TOKENISATION + LABEL ALIGNMENT -------------------------------------------
tok = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)
def align(batch):
    enc = tok(batch["tokens"], is_split_into_words=True,
              truncation=True, max_length=512)
    new_labels = []
    for i, seq in enumerate(batch["ner_tags"]):
        word_ids = enc.word_ids(batch_index=i)
        prev = None
        aligned = []
        for w in word_ids:
            if w is None:
                aligned.append(-100)
            elif w != prev:
                aligned.append(seq[w])
            else:
                aligned.append(-100)
            prev = w
        new_labels.append(aligned)
    enc["labels"] = new_labels
    return enc

train_ds = train_ds.map(align, batched=True, remove_columns=["ner_tags"])
dev_ds   = dev_ds  .map(align, batched=True, remove_columns=["ner_tags"])
test_ref = test_ref.map(align, batched=True, remove_columns=["ner_tags"])

# 7. MODEL, METRICS, TRAINER ---------------------------------------------------
metric = evaluate.load("seqeval")
def compute_metrics(p):
    preds, labs = p
    preds = np.argmax(preds, axis=2)
    true_preds, true_labs = [], []
    for pr, lb in zip(preds, labs):
        pr_l, lb_l = [], []
        for p_i, l_i in zip(pr, lb):
            if l_i != -100:
                pr_l.append(id2label[p_i])
                lb_l.append(id2label[l_i])
        true_preds.append(pr_l)
        true_labs.append(lb_l)
    res = metric.compute(predictions=true_preds, references=true_labs,
                         zero_division=0)
    return {k.replace("overall_", ""): v for k,v in res.items()}

model = AutoModelForTokenClassification.from_pretrained(
    args.model_name, num_labels=num_labels,
    id2label=id2label, label2id=label2id)

train_args = TrainingArguments(
    output_dir     = args.output_dir,
    eval_strategy="epoch",           # ← one eval *after* each epoch
    save_strategy  ="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    num_train_epochs=args.num_train_epochs,
    per_device_train_batch_size=args.per_device_train_batch_size,
    per_device_eval_batch_size=args.per_device_eval_batch_size,
    learning_rate = args.learning_rate,
    weight_decay  = args.weight_decay,
    warmup_steps  = args.warmup_steps,
    seed = args.seed,
    fp16 = torch.cuda.is_available(),
    report_to="none")

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_ds,
    eval_dataset =dev_ds,
    tokenizer=tok,
    data_collator=DataCollatorForTokenClassification(tok),
    compute_metrics=compute_metrics)

# 8. TRAIN & FINAL EVALUATION --------------------------------------------------
log.info("⏳  Starting fine‑tuning …")
trainer.train()
log.info("✅  Finished training.  Best dev‑set F1: %.4f",
         trainer.state.best_metric or -1)

log.info("🏁  Test‑set metrics:")
print(trainer.evaluate(eval_dataset=test_ref))


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Map:   0%|          | 0/8777 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-0eec3401fe9a>:192: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Precision,Recall,F1,Accuracy
1,No log,0.761701,"{'precision': 0.7509477104518925, 'recall': 0.9824064234266147, 'f1': 0.8512234632108449, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.750948,0.915392,0.825056,0.755246
2,0.984600,0.692770,"{'precision': 0.796678765297273, 'recall': 0.9762270240484906, 'f1': 0.8773611602405378, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.796679,0.909634,0.849418,0.801410
3,0.984600,0.628235,"{'precision': 0.8984942570721214, 'recall': 0.8713346715472114, 'f1': 0.884706070415218, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.898494,0.811897,0.853004,0.811146
4,0.581000,0.579840,"{'precision': 0.8786852293031566, 'recall': 0.9290746644625497, 'f1': 0.9031776702186682, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.878685,0.865698,0.872144,0.836954
5,0.581000,0.581730,"{'precision': 0.8437489144405461, 'recall': 0.9559963789506829, 'f1': 0.8963722921356608, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.843720,0.890784,0.866613,0.831701
6,0.461800,0.563691,"{'precision': 0.8604154809334092, 'recall': 0.9520210965481953, 'f1': 0.9039032866832339, 'number': 25407}","{'precision': 0.125, 'recall': 0.00129366106080207, 'f1': 0.0025608194622279133, 'number': 773}","{'precision': 0.5882352941176471, 'recall': 0.00919963201471941, 'f1': 0.018115942028985504, 'number': 1087}",0.860042,0.887483,0.873547,0.842864
7,0.461800,0.571060,"{'precision': 0.885239755558055, 'recall': 0.9293501790845042, 'f1': 0.9067588325652842, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.7122302158273381, 'recall': 0.09107635694572216, 'f1': 0.16150081566068517, 'number': 1087}",0.883914,0.869586,0.876692,0.845776
8,0.377800,0.578216,"{'precision': 0.8779059057583908, 'recall': 0.9378911323650962, 'f1': 0.9069077069457661, 'number': 25407}","{'precision': 0.2, 'recall': 0.009055627425614488, 'f1': 0.017326732673267325, 'number': 773}","{'precision': 0.601593625498008, 'recall': 0.1389144434222631, 'f1': 0.2257100149476831, 'number': 1087}",0.874512,0.879708,0.877103,0.846433
9,0.377800,0.625349,"{'precision': 0.8630107832009081, 'recall': 0.9576101074507025, 'f1': 0.9078527584469859, 'number': 25407}","{'precision': 0.2727272727272727, 'recall': 0.007761966364812419, 'f1': 0.01509433962264151, 'number': 773}","{'precision': 0.5927272727272728, 'recall': 0.1499540018399264, 'f1': 0.23935389133627025, 'number': 1087}",0.859946,0.898485,0.878793,0.847403
10,0.315500,0.575456,"{'precision': 0.8923216034963454, 'recall': 0.9321840437674657, 'f1': 0.9118173593331921, 'number': 25407}","{'precision': 0.16666666666666666, 'recall': 0.031047865459249677, 'f1': 0.05234460196292257, 'number': 773}","{'precision': 0.40589569160997735, 'recall': 0.32934682612695493, 'f1': 0.3636363636363637, 'number': 1087}",0.872969,0.882605,0.877761,0.846433


model.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

{'eval_loss': 0.47220128774642944, 'eval_LOC': {'precision': 0.9475890985324947, 'recall': 0.9094567404426559, 'f1': 0.9281314168377824, 'number': 497}, 'eval_ORG': {'precision': 0.4, 'recall': 0.2222222222222222, 'f1': 0.2857142857142857, 'number': 9}, 'eval_PER': {'precision': 1.0, 'recall': 0.09090909090909091, 'f1': 0.16666666666666669, 'number': 11}, 'eval_precision': 0.9420289855072463, 'eval_recall': 0.8800773694390716, 'eval_f1': 0.9099999999999999, 'eval_accuracy': 0.8861940298507462, 'eval_runtime': 0.0976, 'eval_samples_per_second': 522.305, 'eval_steps_per_second': 20.483, 'epoch': 20.0}


In [ ]:
###############################################################################
# Indic‑NER fine‑tuning on Naamapadam (+ optional custom JSON)
###############################################################################
import argparse, random, json, os, itertools, logging, collections
import numpy as np, torch
from datasets import load_dataset, concatenate_datasets, DatasetDict, Sequence, Value
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)

# ──────────────────────────────────────────────────────────────
# 0.  LOGGING
# ──────────────────────────────────────────────────────────────
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%H:%M:%S",
)
log = logging.getLogger(__name__)

# ──────────────────────────────────────────────────────────────
# 1.  ARGUMENTS & SEEDING
# ──────────────────────────────────────────────────────────────
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, default="ai4bharat/indic-bert")
parser.add_argument("--languages", nargs="+", default=["as"])
parser.add_argument("--custom_data_path", type=str, default=None,
                    help="Path to your extra JSON file (optional)")
parser.add_argument("--output_dir", type=str,
                    default="./indicner-finetuned-naamapadam")
parser.add_argument("--num_train_epochs", type=int, default=5)
parser.add_argument("--per_device_train_batch_size", type=int, default=32)
parser.add_argument("--per_device_eval_batch_size", type=int, default=32)
parser.add_argument("--learning_rate", type=float, default=3e-5)
parser.add_argument("--weight_decay",  type=float, default=0.01)
parser.add_argument("--warmup_steps",  type=int, default=500)
parser.add_argument("--seed", type=int, default=42)
args, _ = parser.parse_known_args()

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
    log.info("CUDA device: %s", torch.cuda.get_device_name(0))
else:
    log.info("CUDA not available – falling back to CPU")

lang = args.languages[0]          # single‑language run

# ──────────────────────────────────────────────────────────────
# 2.  CANONICAL LABEL SET  (Naamapadam has exactly 7)
# ──────────────────────────────────────────────────────────────
label_list = ["B-LOC","B-ORG","B-PER","I-LOC","I-ORG","I-PER","O"]
id2label   = {i:l for i,l in enumerate(label_list)}  # type: ignore
label2id   = {l:i for i,l in id2label.items()}
num_labels = len(label_list)
log.info("Canonical labels: %s", id2label)

# Helper – map *any* tag outside this list to “O”
def normalise_tag(tag: str) -> str:
    tag = str(tag)
    if tag in ("B-PERSON", "I-PERSON"):         # your PERSON alias
        tag = tag.replace("PERSON", "PER")
    return tag if tag in label2id else "O"

# ──────────────────────────────────────────────────────────────
# 3.  LOAD NAAMAPADAM (train / test)
# ──────────────────────────────────────────────────────────────
train_ref = load_dataset("ai4bharat/naamapadam", lang, split="train")
test_ref  = load_dataset("ai4bharat/naamapadam", lang, split="test")

# NEW  ➜  make their `ner_tags` column a plain int sequence
int_seq = Sequence(Value("int64"))
train_ref = train_ref.cast_column("ner_tags", int_seq)
test_ref  = test_ref.cast_column("ner_tags", int_seq)

# ──────────────────────────────────────────────────────────────
# 4.  OPTIONAL: LOAD + CLEAN YOUR CUSTOM JSON
# ──────────────────────────────────────────────────────────────
if args.custom_data_path:
    log.info("Loading custom JSON: %s", args.custom_data_path)
    custom = load_dataset("json", data_files=args.custom_data_path,
                          split="train")

    # (a) ensure tokens / ner_tags length match
    broken = [i for i, ex in enumerate(custom)
              if len(ex["tokens"]) != len(ex["ner_tags"])]
    if broken:
        log.warning("⚠️  %d rows have mismatching lengths – they’ll be dropped",
                    len(broken))
        custom = custom.select([i for i in range(len(custom)) if i not in broken])

    # (b) normalise & map to ints
    def _clean(batch):
        out = []
        for tags in batch["ner_tags"]:
            out.append([label2id[normalise_tag(t)] for t in tags])
        batch["ner_tags"] = out
        return batch

    custom = custom.map(_clean, batched=True)
    log.info("Custom set after cleaning: %d sentences", len(custom))
    train_all = concatenate_datasets([train_ref, custom])
else:
    train_all = train_ref

# quick label‑distribution print‑out
def label_hist(ds, name):
    flat = list(itertools.chain.from_iterable(ds["ner_tags"]))
    c = collections.Counter(flat)
    log.info("%s label distribution: %s",
             name, {id2label[k]: v for k,v in c.items()})
label_hist(train_all, "TRAIN")
label_hist(test_ref,  "TEST ")

# 5. TRAIN / DEV SPLIT ---------------------------------------------------------
split = train_all.train_test_split(train_size=0.75, seed=args.seed)
train_ds, dev_ds = split["train"], split["test"]
log.info("Train %d  |  Dev %d  |  Test %d", len(train_ds), len(dev_ds), len(test_ref))

# 6. TOKENISATION + LABEL ALIGNMENT -------------------------------------------
tok = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)
def align(batch):
    enc = tok(batch["tokens"], is_split_into_words=True,
              truncation=True, max_length=512)
    new_labels = []
    for i, seq in enumerate(batch["ner_tags"]):
        word_ids = enc.word_ids(batch_index=i)
        prev = None
        aligned = []
        for w in word_ids:
            if w is None:
                aligned.append(-100)
            elif w != prev:
                aligned.append(seq[w])
            else:
                aligned.append(-100)
            prev = w
        new_labels.append(aligned)
    enc["labels"] = new_labels
    return enc

train_ds = train_ds.map(align, batched=True, remove_columns=["ner_tags"])
dev_ds   = dev_ds  .map(align, batched=True, remove_columns=["ner_tags"])
test_ref = test_ref.map(align, batched=True, remove_columns=["ner_tags"])

# 7. MODEL, METRICS, TRAINER ---------------------------------------------------
metric = evaluate.load("seqeval")
def compute_metrics(p):
    preds, labs = p
    preds = np.argmax(preds, axis=2)
    true_preds, true_labs = [], []
    for pr, lb in zip(preds, labs):
        pr_l, lb_l = [], []
        for p_i, l_i in zip(pr, lb):
            if l_i != -100:
                pr_l.append(id2label[p_i])
                lb_l.append(id2label[l_i])
        true_preds.append(pr_l)
        true_labs.append(lb_l)
    res = metric.compute(predictions=true_preds, references=true_labs,
                         zero_division=0)
    return {k.replace("overall_", ""): v for k,v in res.items()}

model = AutoModelForTokenClassification.from_pretrained(
    args.model_name, num_labels=num_labels,
    id2label=id2label, label2id=label2id)

train_args = TrainingArguments(
    output_dir     = args.output_dir,
    eval_strategy="epoch",           # ← one eval *after* each epoch
    save_strategy  ="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    num_train_epochs=args.num_train_epochs,
    per_device_train_batch_size=args.per_device_train_batch_size,
    per_device_eval_batch_size=args.per_device_eval_batch_size,
    learning_rate = args.learning_rate,
    weight_decay  = args.weight_decay,
    warmup_steps  = args.warmup_steps,
    seed = args.seed,
    fp16 = torch.cuda.is_available(),
    report_to="none")

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_ds,
    eval_dataset =dev_ds,
    tokenizer=tok,
    data_collator=DataCollatorForTokenClassification(tok),
    compute_metrics=compute_metrics)

# 8. TRAIN & FINAL EVALUATION --------------------------------------------------
log.info("⏳  Starting fine‑tuning …")
trainer.train()
log.info("✅  Finished training.  Best dev‑set F1: %.4f",
         trainer.state.best_metric or -1)

log.info("🏁  Test‑set metrics:")
print(trainer.evaluate(eval_dataset=test_ref))


Map:   0%|          | 0/2567 [00:00<?, ? examples/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-cc99a0954ec2>:192: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Precision,Recall,F1,Accuracy
1,No log,0.390798,"{'precision': 0.9141859544292862, 'recall': 0.9885563037639641, 'f1': 0.9499177139437462, 'number': 25691}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 523}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 704}",0.914186,0.943495,0.928609,0.917318
2,No log,0.320645,"{'precision': 0.9175696312995418, 'recall': 0.9822505935930871, 'f1': 0.9488090538228714, 'number': 25691}","{'precision': 0.23076923076923078, 'recall': 0.0057361376673040155, 'f1': 0.011194029850746268, 'number': 523}","{'precision': 0.9090909090909091, 'recall': 0.014204545454545454, 'f1': 0.027972027972027972, 'number': 704}",0.917242,0.937960,0.927485,0.919045
3,0.544400,0.267844,"{'precision': 0.9242165034761403, 'recall': 0.9883227589428204, 'f1': 0.9551952449025657, 'number': 25691}","{'precision': 0.48, 'recall': 0.045889101338432124, 'f1': 0.08376963350785341, 'number': 523}","{'precision': 0.8434782608695652, 'recall': 0.1377840909090909, 'f1': 0.23687423687423687, 'number': 704}",0.923077,0.947767,0.935259,0.926245
4,0.544400,0.249893,"{'precision': 0.951655881233346, 'recall': 0.9731034214316298, 'f1': 0.9622601566559535, 'number': 25691}","{'precision': 0.4293193717277487, 'recall': 0.3135755258126195, 'f1': 0.36243093922651937, 'number': 523}","{'precision': 0.5741029641185648, 'recall': 0.5227272727272727, 'f1': 0.5472118959107807, 'number': 704}",0.935478,0.948510,0.941949,0.933228
5,0.236200,0.245809,"{'precision': 0.9505841430677908, 'recall': 0.9786306488653614, 'f1': 0.9644035289604909, 'number': 25691}","{'precision': 0.5506756756756757, 'recall': 0.31166347992351817, 'f1': 0.398046398046398, 'number': 523}","{'precision': 0.5968503937007874, 'recall': 0.5383522727272727, 'f1': 0.5660941000746826, 'number': 704}",0.938057,0.954157,0.946039,0.937907


{'eval_loss': 0.24383287131786346, 'eval_LOC': {'precision': 0.9393346379647749, 'recall': 0.96579476861167, 'f1': 0.9523809523809522, 'number': 497}, 'eval_ORG': {'precision': 0.16666666666666666, 'recall': 0.1111111111111111, 'f1': 0.13333333333333333, 'number': 9}, 'eval_PER': {'precision': 0.3333333333333333, 'recall': 0.2727272727272727, 'f1': 0.3, 'number': 11}, 'eval_precision': 0.9201520912547528, 'eval_recall': 0.9361702127659575, 'eval_f1': 0.9280920421860019, 'eval_accuracy': 0.9291044776119403, 'eval_runtime': 0.0887, 'eval_samples_per_second': 574.886, 'eval_steps_per_second': 22.545, 'epoch': 5.0}


In [3]:
###############################################################################
# Indic-NER fine-tuning on Naamapadam (+ optional custom JSON)
# with hyperparameter tuning
###############################################################################
import argparse, random, json, os, itertools, logging, collections
import numpy as np, torch
from datasets import load_dataset, concatenate_datasets, DatasetDict, Sequence, Value
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)

# ──────────────────────────────────────────────────────────────
# 0.  LOGGING
# ──────────────────────────────────────────────────────────────
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%H:%M:%S",
)
log = logging.getLogger(__name__)

# ──────────────────────────────────────────────────────────────
# 1.  ARGUMENTS & SEEDING
# ──────────────────────────────────────────────────────────────
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, default="ai4bharat/indic-bert")
parser.add_argument("--languages", nargs="+", default=["as"])
parser.add_argument("--custom_data_path", type=str, default="/content/drive/MyDrive/Yash_final_btp/naamapadam_proj/0.6/0.4/naamapadam_assamese.json",
                    help="Path to your extra JSON file (optional)")
parser.add_argument("--output_dir", type=str,
                    default="./indicner-finetuned-naamapadam")
parser.add_argument("--num_train_epochs", type=int, default=10)
parser.add_argument("--per_device_train_batch_size", type=int, default=32)
parser.add_argument("--per_device_eval_batch_size", type=int, default=32)
parser.add_argument("--learning_rate", type=float, default=1e-5)
parser.add_argument("--weight_decay",  type=float, default=0.01)
parser.add_argument("--warmup_steps",  type=int, default=500)
parser.add_argument("--seed", type=int, default=42)
args, _ = parser.parse_known_args()

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
    log.info("CUDA device: %s", torch.cuda.get_device_name(0))
else:
    log.info("CUDA not available – falling back to CPU")

lang = args.languages[0]

# ──────────────────────────────────────────────────────────────
# 2.  CANONICAL LABEL SET  (Naamapadam has exactly 7)
# ──────────────────────────────────────────────────────────────
label_list = ["B-LOC","B-ORG","B-PER","I-LOC","I-ORG","I-PER","O"]
id2label   = {i:l for i,l in enumerate(label_list)}  # type: ignore
label2id   = {l:i for i,l in id2label.items()}
num_labels = len(label_list)
log.info("Canonical labels: %s", id2label)

# Helper – map *any* tag outside this list to “O”
def normalise_tag(tag: str) -> str:
    tag = str(tag)
    if tag in ("B-PERSON", "I-PERSON"):         # your PERSON alias
        tag = tag.replace("PERSON", "PER")
    return tag if tag in label2id else "O"

# ──────────────────────────────────────────────────────────────
# 3.  LOAD NAAMAPADAM (train / test)
# ──────────────────────────────────────────────────────────────
train_ref = load_dataset("ai4bharat/naamapadam", lang, split="train")
test_ref  = load_dataset("ai4bharat/naamapadam", lang, split="test")

# NEW  ➜  make their `ner_tags` column a plain int sequence
int_seq = Sequence(Value("int64"))
train_ref = train_ref.cast_column("ner_tags", int_seq)
test_ref  = test_ref.cast_column("ner_tags", int_seq)

# ──────────────────────────────────────────────────────────────
# 4.  OPTIONAL: LOAD + CLEAN YOUR CUSTOM JSON
# ──────────────────────────────────────────────────────────────
if args.custom_data_path:
    log.info("Loading custom JSON: %s", args.custom_data_path)
    custom = load_dataset("json", data_files=args.custom_data_path,
                          split="train")

    # (a) ensure tokens / ner_tags length match
    broken = [i for i, ex in enumerate(custom)
              if len(ex["tokens"]) != len(ex["ner_tags"])]
    if broken:
        log.warning("⚠️  %d rows have mismatching lengths – they’ll be dropped",
                    len(broken))
        custom = custom.select([i for i in range(len(custom)) if i not in broken])

    # (b) normalise & map to ints
    def _clean(batch):
        out = []
        for tags in batch["ner_tags"]:
            out.append([label2id[normalise_tag(t)] for t in tags])
        batch["ner_tags"] = out
        return batch

    custom = custom.map(_clean, batched=True)
    log.info("Custom set after cleaning: %d sentences", len(custom))
    train_all = concatenate_datasets([train_ref, custom])
else:
    train_all = train_ref

# quick label-distribution print-out
def label_hist(ds, name):
    flat = list(itertools.chain.from_iterable(ds["ner_tags"]))
    c = collections.Counter(flat)
    log.info("%s label distribution: %s",
             name, {id2label[k]: v for k,v in c.items()})
label_hist(train_all, "TRAIN")
label_hist(test_ref,  "TEST ")

# ──────────────────────────────────────────────────────────────
# 5. TRAIN / DEV SPLIT
# ──────────────────────────────────────────────────────────────
split = train_all.train_test_split(train_size=0.75, seed=args.seed)
train_ds, dev_ds = split["train"], split["test"]
log.info("Train %d  |  Dev %d  |  Test %d",
         len(train_ds), len(dev_ds), len(test_ref))

# ──────────────────────────────────────────────────────────────
# 6. TOKENISATION + LABEL ALIGNMENT
# ──────────────────────────────────────────────────────────────
tok = AutoTokenizer.from_pretrained(args.model_name, use_fast=True)
def align(batch):
    enc = tok(batch["tokens"], is_split_into_words=True,
              truncation=True, max_length=512)
    new_labels = []
    for i, seq in enumerate(batch["ner_tags"]):
        word_ids = enc.word_ids(batch_index=i)
        prev = None
        aligned = []
        for w in word_ids:
            if w is None:
                aligned.append(-100)
            elif w != prev:
                aligned.append(seq[w])
            else:
                aligned.append(-100)
            prev = w
        new_labels.append(aligned)
    enc["labels"] = new_labels
    return enc

train_ds = train_ds.map(align, batched=True, remove_columns=["ner_tags"])
dev_ds   = dev_ds  .map(align, batched=True, remove_columns=["ner_tags"])
test_ref = test_ref.map(align, batched=True, remove_columns=["ner_tags"])

# ──────────────────────────────────────────────────────────────
# 7. MODEL, METRICS, TRAINER
# ──────────────────────────────────────────────────────────────
metric = evaluate.load("seqeval")
def compute_metrics(p):
    preds, labs = p
    preds = np.argmax(preds, axis=2)
    true_preds, true_labs = [], []
    for pr, lb in zip(preds, labs):
        pr_l, lb_l = [], []
        for p_i, l_i in zip(pr, lb):
            if l_i != -100:
                pr_l.append(id2label[p_i])
                lb_l.append(id2label[l_i])
        true_preds.append(pr_l)
        true_labs.append(lb_l)
    res = metric.compute(predictions=true_preds, references=true_labs,
                         zero_division=0)
    return {k.replace("overall_", ""): v for k,v in res.items()}

def model_init():
    return AutoModelForTokenClassification.from_pretrained(
        args.model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id
    )

train_args = TrainingArguments(
    output_dir     = args.output_dir,
    eval_strategy  ="epoch",
    save_strategy  ="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    num_train_epochs=args.num_train_epochs,
    per_device_train_batch_size=args.per_device_train_batch_size,
    per_device_eval_batch_size=args.per_device_eval_batch_size,
    learning_rate = args.learning_rate,
    weight_decay  = args.weight_decay,
    warmup_steps  = args.warmup_steps,
    seed = args.seed,
    fp16 = torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model_init=model_init,
    args=train_args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tok,
    data_collator=DataCollatorForTokenClassification(tok),
    compute_metrics=compute_metrics)

# ──────────────────────────────────────────────────────────────
# 7b.  OPTIONAL: HYPERPARAMETER TUNING (Optuna)
# ──────────────────────────────────────────────────────────────
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=10,
    hp_space=lambda _: {
        "learning_rate": np.random.uniform(1e-6, 1e-4),
        "per_device_train_batch_size": np.random.choice([16, 32]),
        "weight_decay": np.random.uniform(0.0, 0.3),
    },
    compute_objective=lambda metrics: metrics["eval_f1"],
)
log.info("Best hyperparameters found: %s", best_run)

# ──────────────────────────────────────────────────────────────
# 8. TRAIN & FINAL EVALUATION
# ──────────────────────────────────────────────────────────────
log.info("⏳  Starting fine-tuning …")
trainer.train()
log.info("✅  Finished training.  Best dev-set F1: %.4f",
         trainer.state.best_metric or -1)

log.info("🏁  Test-set metrics:")
print(trainer.evaluate(eval_dataset=test_ref))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

naamapadam.py:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/654k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

...

[I 2025-05-11 21:53:06,824] Trial 8 pruned. 
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Precision,Recall,F1,Accuracy
1,0.858400,0.607546,"{'precision': 0.8004271015336828, 'recall': 0.9736686739874838, 'f1': 0.8785893133024346, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.4, 'recall': 0.0018399264029438822, 'f1': 0.003663003663003663, 'number': 1087}",0.800362,0.907324,0.850493,0.802981
2,0.528900,0.571531,"{'precision': 0.8449750901906888, 'recall': 0.9679615853898532, 'f1': 0.9022967420017611, 'number': 25407}","{'precision': 0.14736842105263157, 'recall': 0.018111254851228976, 'f1': 0.03225806451612903, 'number': 773}","{'precision': 0.3997308209959623, 'recall': 0.2732290708371665, 'f1': 0.32459016393442625, 'number': 1087}",0.831714,0.913338,0.870617,0.833586
3,0.409200,0.532064,"{'precision': 0.9220736876426475, 'recall': 0.8904632581572007, 'f1': 0.9059928318282842, 'number': 25407}","{'precision': 0.5229885057471264, 'recall': 0.11772315653298836, 'f1': 0.19218585005279828, 'number': 773}","{'precision': 0.636568848758465, 'recall': 0.2594296228150874, 'f1': 0.3686274509803922, 'number': 1087}",0.914285,0.843400,0.877413,0.842978


[I 2025-05-11 21:55:13,653] Trial 9 pruned. 
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Precision,Recall,F1,Accuracy
1,0.844100,0.652718,"{'precision': 0.7668900227454355, 'recall': 0.9820128311095367, 'f1': 0.8612208971194836, 'number': 25407}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 773}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1087}",0.766890,0.915025,0.834434,0.774831
2,0.541900,0.616695,"{'precision': 0.8228768806669102, 'recall': 0.97516432479238, 'f1': 0.892571510915772, 'number': 25407}","{'precision': 0.11764705882352941, 'recall': 0.00517464424320828, 'f1': 0.009913258983890954, 'number': 773}","{'precision': 0.47161572052401746, 'recall': 0.19871205151793928, 'f1': 0.2796116504854369, 'number': 1087}",0.816836,0.916713,0.863897,0.822194
3,0.422800,0.538142,"{'precision': 0.9288302169658101, 'recall': 0.8778683040107057, 'f1': 0.9026305139619587, 'number': 25407}","{'precision': 0.4018264840182648, 'recall': 0.11384217335058215, 'f1': 0.1774193548387097, 'number': 773}","{'precision': 0.603448275862069, 'recall': 0.3219871205151794, 'f1': 0.4199160167966407, 'number': 1087}",0.916573,0.834048,0.873365,0.838496
4,0.334900,0.560640,"{'precision': 0.9061983629865887, 'recall': 0.9281694021332704, 'f1': 0.9170523041026638, 'number': 25407}","{'precision': 0.5158730158730159, 'recall': 0.08408796895213454, 'f1': 0.14460511679644047, 'number': 773}","{'precision': 0.7345679012345679, 'recall': 0.21895124195032198, 'f1': 0.3373493975903614, 'number': 1087}",0.902240,0.875967,0.888910,0.858424
5,0.265800,0.566423,"{'precision': 0.8924022346368715, 'recall': 0.9430865509505254, 'f1': 0.9170446064642058, 'number': 25407}","{'precision': 0.4358974358974359, 'recall': 0.1539456662354463, 'f1': 0.22753346080305925, 'number': 773}","{'precision': 0.6811320754716981, 'recall': 0.33210671573137074, 'f1': 0.44650587507730366, 'number': 1087}",0.883846,0.896358,0.890058,0.858509
6,0.234500,0.571079,"{'precision': 0.8915867158671587, 'recall': 0.9509977565237927, 'f1': 0.9203344315995963, 'number': 25407}","{'precision': 0.44126074498567336, 'recall': 0.19922380336351875, 'f1': 0.2745098039215686, 'number': 773}","{'precision': 0.5455621301775148, 'recall': 0.42410303587856485, 'f1': 0.4772256728778468, 'number': 1087}",0.875698,0.908681,0.891885,0.863334
7,0.184100,0.581591,"{'precision': 0.9, 'recall': 0.9454481048529932, 'f1': 0.922164424055128, 'number': 25407}","{'precision': 0.4564102564102564, 'recall': 0.23027166882276842, 'f1': 0.30610490111779876, 'number': 773}","{'precision': 0.5581683168316832, 'recall': 0.41490340386384544, 'f1': 0.47598944591029024, 'number': 1087}",0.883893,0.904023,0.893845,0.866417
8,0.167400,0.626232,"{'precision': 0.890546528803545, 'recall': 0.949187231865234, 'f1': 0.9189323070474594, 'number': 25407}","{'precision': 0.4376470588235294, 'recall': 0.240620957309185, 'f1': 0.31051752921535897, 'number': 773}","{'precision': 0.5606595995288575, 'recall': 0.43790248390064396, 'f1': 0.4917355371900826, 'number': 1087}",0.873880,0.908717,0.890958,0.860593
9,0.142900,0.631531,"{'precision': 0.8950543274634695, 'recall': 0.9402526862675641, 'f1': 0.9170969537593335, 'number': 25407}","{'precision': 0.39723320158102765, 'recall': 0.26002587322121606, 'f1': 0.3143080531665363, 'number': 773}","{'precision': 0.536036036036036, 'recall': 0.43790248390064396, 'f1': 0.48202531645569624, 'number': 1087}",0.874733,0.900943,0.887644,0.858224
10,0.119600,0.638251,"{'precision': 0.902084912812737, 'recall': 0.9366316369504467, 'f1': 0.9190337343348717, 'number': 25407}","{'precision': 0.390715667311412, 'recall': 0.2613195342820181, 'f1': 0.3131782945736434, 'number': 773}","{'precision': 0.5383734249713631, 'recall': 0.43238270469181234, 'f1': 0.47959183673469385, 'number': 1087}",0.881131,0.897385,0.889184,0.860536


{'eval_loss': 0.23289106786251068, 'eval_LOC': {'precision': 0.952286282306163, 'recall': 0.9637826961770624, 'f1': 0.9580000000000001, 'number': 497}, 'eval_ORG': {'precision': 0.3333333333333333, 'recall': 0.2222222222222222, 'f1': 0.26666666666666666, 'number': 9}, 'eval_PER': {'precision': 0.5555555555555556, 'recall': 0.45454545454545453, 'f1': 0.5, 'number': 11}, 'eval_precision': 0.9382239382239382, 'eval_recall': 0.9400386847195358, 'eval_f1': 0.9391304347826087, 'eval_accuracy': 0.9402985074626866, 'eval_runtime': 0.1032, 'eval_samples_per_second': 494.166, 'eval_steps_per_second': 19.379, 'epoch': 10.0}
